## Лабораторная работа №2. Обработка признаков
**Задание:**

Выбрать набор данных (датасет), содержащий категориальные и числовые признаки и пропуски в данных. Для выполнения следующих пунктов можно использовать несколько различных наборов данных (один для обработки пропусков, другой для категориальных признаков и т.д.).
Для выбранного датасета (датасетов) на основе материалов лекций решить следующие задачи:
1. устранение пропусков в данных;
1. кодирование категориальных признаков;
1. нормализацию числовых признаков.

In [56]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from category_encoders.count import CountEncoder as ce_CountEncoder

In [57]:
data_loaded = pd.read_csv('world-happiness-report-2021.csv', sep=',')
data = data_loaded.rename(columns={
})
data.head()

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.0,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253
1,Denmark,Western Europe,NaN,0.035,7.687,7.552,10.933,0.954,72.7,0.946,0.030,0.179,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868
2,Switzerland,Western Europe,7.571,0.036,7.643,7.500,11.117,0.942,74.4,0.919,0.025,0.292,2.43,1.566,1.079,0.816,0.653,0.204,0.413,2.839
3,Iceland,Western Europe,7.554,0.059,7.670,7.438,10.878,0.983,73.0,0.955,0.160,0.673,2.43,1.482,1.172,0.772,0.698,0.293,0.170,2.967
4,Netherlands,Western Europe,7.464,0.027,7.518,7.410,10.932,0.942,72.4,0.913,0.175,0.338,2.43,1.501,1.079,0.753,0.647,0.302,0.384,2.798


In [58]:
to_delete = []
data=data.drop(to_delete, axis=1)
data_features = list(zip(
# признаки
[i for i in data.columns],
zip(
 # типы колонок
 [str(i) for i in data.dtypes],
 # проверим есть ли пропущенные значения
 [i for i in data.isnull().sum()]
)))
# Признаки с типом данных и количеством пропусков
data_features


[('Country name', ('object', 0)),
 ('Regional indicator', ('object', 0)),
 ('Ladder score', ('float64', 4)),
 ('Standard error of ladder score', ('float64', 0)),
 ('upperwhisker', ('float64', 0)),
 ('lowerwhisker', ('float64', 0)),
 ('Logged GDP per capita', ('float64', 0)),
 ('Social support', ('float64', 0)),
 ('Healthy life expectancy', ('float64', 0)),
 ('Freedom to make life choices', ('float64', 0)),
 ('Generosity', ('float64', 0)),
 ('Perceptions of corruption', ('float64', 0)),
 ('Ladder score in Dystopia', ('float64', 0)),
 ('Explained by: Log GDP per capita', ('float64', 0)),
 ('Explained by: Social support', ('float64', 0)),
 ('Explained by: Healthy life expectancy', ('float64', 0)),
 ('Explained by: Freedom to make life choices', ('float64', 10)),
 ('Explained by: Generosity', ('float64', 0)),
 ('Explained by: Perceptions of corruption', ('float64', 0)),
 ('Dystopia + residual', ('float64', 0))]

#### Устранение пропусков данных

In [59]:
cols_with_na = [c for c in data.columns if data[c].isnull().sum() > 0]
[(c, data[c].isnull().sum(), "%.3f" % data[c].isnull().mean()) for c in cols_with_na]

[('Ladder score', 4, '0.027'),
 ('Explained by: Freedom to make life choices', 10, '0.067')]

In [62]:
cols_to_delete = []
data_droped = data.drop(cols_to_delete, axis=1)
data_droped

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.000,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253
1,Denmark,Western Europe,NaN,0.035,7.687,7.552,10.933,0.954,72.700,0.946,0.030,0.179,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868
2,Switzerland,Western Europe,7.571,0.036,7.643,7.500,11.117,0.942,74.400,0.919,0.025,0.292,2.43,1.566,1.079,0.816,0.653,0.204,0.413,2.839
3,Iceland,Western Europe,7.554,0.059,7.670,7.438,10.878,0.983,73.000,0.955,0.160,0.673,2.43,1.482,1.172,0.772,0.698,0.293,0.170,2.967
4,Netherlands,Western Europe,7.464,0.027,7.518,7.410,10.932,0.942,72.400,0.913,0.175,0.338,2.43,1.501,1.079,0.753,0.647,0.302,0.384,2.798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Lesotho,Sub-Saharan Africa,3.512,0.120,3.748,3.276,7.926,0.787,48.700,0.715,-0.131,0.915,2.43,0.451,0.731,0.007,0.405,0.103,0.015,1.800
145,Botswana,Sub-Saharan Africa,3.467,0.074,3.611,3.322,9.782,0.784,59.269,0.824,-0.246,0.801,2.43,1.099,0.724,0.340,0.539,0.027,0.088,0.648
146,Rwanda,Sub-Saharan Africa,3.415,0.068,3.548,3.282,7.676,0.552,61.400,0.897,0.061,0.167,2.43,0.364,0.202,0.407,0.627,0.227,0.493,1.095
147,Zimbabwe,Sub-Saharan Africa,3.145,0.058,3.259,3.030,7.943,0.750,56.201,0.677,-0.047,0.821,2.43,0.457,0.649,0.243,0.359,0.157,0.075,1.205


In [63]:
data_to_impute = data_droped[['Ladder score','Explained by: Freedom to make life choices']]
data_to_impute.isnull().sum()

Ladder score                                   4
Explained by: Freedom to make life choices    10
dtype: int64

In [64]:
knnimputer = KNNImputer(
    n_neighbors=5, 
    weights='distance', 
    metric='nan_euclidean', 
    add_indicator=False, 
)
array_imputed = knnimputer.fit_transform(data_to_impute)
data_imputed = data_droped.merge(pd.DataFrame(array_imputed, columns=data_to_impute.columns))
data_imputed.isnull().sum()

Country name                                  0
Regional indicator                            0
Ladder score                                  0
Standard error of ladder score                0
upperwhisker                                  0
lowerwhisker                                  0
Logged GDP per capita                         0
Social support                                0
Healthy life expectancy                       0
Freedom to make life choices                  0
Generosity                                    0
Perceptions of corruption                     0
Ladder score in Dystopia                      0
Explained by: Log GDP per capita              0
Explained by: Social support                  0
Explained by: Healthy life expectancy         0
Explained by: Freedom to make life choices    0
Explained by: Generosity                      0
Explained by: Perceptions of corruption       0
Dystopia + residual                           0
dtype: int64

#### Кодирование категориальных признаков

In [65]:
print(data_imputed["Regional indicator"].unique(), '\n')

encoder_test = ce_CountEncoder()
col_encoded_test = encoder_test.fit_transform(data_imputed[['Regional indicator']])
print(col_encoded_test["Regional indicator"].unique(), '\n')

encoder = ce_CountEncoder(normalize=True)
col_encoded = encoder.fit_transform(data_imputed[['Regional indicator']])
print(col_encoded["Regional indicator"].unique())

['Western Europe' 'North America and ANZ' 'Middle East and North Africa'
 'Latin America and Caribbean' 'Central and Eastern Europe' 'East Asia'
 'Southeast Asia' 'Commonwealth of Independent States'
 'Sub-Saharan Africa' 'South Asia'] 

[21  4 17 14  5  9 10 34  7] 

[0.15217391 0.02898551 0.12318841 0.10144928 0.03623188 0.06521739
 0.07246377 0.24637681 0.05072464]


In [66]:
data_encoded = data_imputed.copy()
data_encoded['Regional indicator'] = col_encoded['Regional indicator']
data_encoded.head()

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,0.152174,7.842,0.032,7.904,7.780,10.775,0.954,72.0,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253
1,Switzerland,0.152174,7.571,0.036,7.643,7.500,11.117,0.942,74.4,0.919,0.025,0.292,2.43,1.566,1.079,0.816,0.653,0.204,0.413,2.839
2,Iceland,0.152174,7.554,0.059,7.670,7.438,10.878,0.983,73.0,0.955,0.160,0.673,2.43,1.482,1.172,0.772,0.698,0.293,0.170,2.967
3,Netherlands,0.152174,7.464,0.027,7.518,7.410,10.932,0.942,72.4,0.913,0.175,0.338,2.43,1.501,1.079,0.753,0.647,0.302,0.384,2.798
4,Norway,0.152174,7.392,0.035,7.462,7.323,11.053,0.954,73.3,0.960,0.093,0.270,2.43,1.543,1.108,0.782,0.703,0.249,0.427,2.580


#### Нормализация числовых признаков

In [67]:
cols_to_scale = ["Ladder score", "Standard error of ladder score", "upperwhisker", "lowerwhisker", "Logged GDP per capita", "Social support", "Healthy life expectancy", "Freedom to make life choices", "Generosity", "Perceptions of corruption", "Ladder score in Dystopia"]

MMScaler = MinMaxScaler()
data_scaled = data_encoded.copy()
for col in cols_to_scale:
    data_scaled[col] = MMScaler.fit_transform(data_encoded[[col]])
data_scaled.describe()

,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
count,138.000000,138.000000,138.000000,138.000000,138.000000,138.000000,138.000000,138.000000,138.000000,138.000000,138.000000,138.0,138.000000,138.000000,138.000000,138.000000,138.000000,138.000000,138.000000
mean,0.138731,0.559743,0.221434,0.568775,0.550828,0.558635,0.672450,0.577103,0.703024,0.331212,0.757327,0.0,0.978087,0.788630,0.517594,0.494768,0.179370,0.132862,2.409000
std,0.070084,0.205781,0.149712,0.202502,0.209259,0.232779,0.221374,0.240599,0.199294,0.183110,0.205587,0.0,0.407566,0.259389,0.215834,0.140161,0.099128,0.112424,0.534394
min,0.028986,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.648000
25%,0.079710,0.428652,0.117347,0.446260,0.415166,0.379838,0.546154,0.384592,0.579585,0.190663,0.692240,0.0,0.665250,0.640250,0.345000,0.408000,0.103500,0.059250,2.140500
50%,0.123188,0.554334,0.193878,0.556424,0.551679,0.586193,0.706731,0.636488,0.718858,0.306024,0.823804,0.0,1.026500,0.829000,0.571000,0.506000,0.166000,0.096500,2.477000
75%,0.152174,0.712916,0.299320,0.719292,0.703902,0.755387,0.846154,0.744917,0.856401,0.445482,0.892065,0.0,1.323000,0.992750,0.667750,0.603000,0.241000,0.168500,2.784000
max,0.246377,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.751000,1.172000,0.897000,0.703000,0.541000,0.547000,3.482000


#### Итоговые данные

In [68]:
data_scaled

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,0.152174,1.000000,0.040816,1.000000,1.000000,0.826018,0.944231,0.826058,0.980969,0.228916,0.121354,0.0,1.446,1.106,0.741,0.691,0.124,0.481,3.253
1,Switzerland,0.152174,0.949051,0.068027,0.950829,0.947477,0.894254,0.921154,0.910342,0.929066,0.377108,0.245041,0.0,1.566,1.079,0.816,0.653,0.204,0.413,2.839
2,Iceland,0.152174,0.945854,0.224490,0.955916,0.935847,0.846568,1.000000,0.861176,0.991349,0.539759,0.689615,0.0,1.482,1.172,0.772,0.698,0.293,0.170,2.967
3,Netherlands,0.152174,0.928934,0.006803,0.927280,0.930595,0.857342,0.921154,0.840105,0.918685,0.557831,0.298716,0.0,1.501,1.079,0.753,0.647,0.302,0.384,2.798
4,Norway,0.152174,0.915398,0.061224,0.916729,0.914275,0.881484,0.944231,0.871712,1.000000,0.459036,0.219370,0.0,1.543,1.108,0.782,0.703,0.249,0.427,2.580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,Lesotho,0.246377,0.185937,0.639456,0.217031,0.155130,0.257582,0.623077,0.007796,0.576125,0.189157,0.971995,0.0,0.451,0.731,0.007,0.405,0.103,0.015,1.800
134,Botswana,0.246377,0.177477,0.326531,0.191221,0.163759,0.627893,0.617308,0.378964,0.764706,0.050602,0.838973,0.0,1.099,0.724,0.340,0.539,0.027,0.088,0.648
135,Rwanda,0.246377,0.167701,0.285714,0.179352,0.156256,0.207702,0.171154,0.453802,0.891003,0.420482,0.099183,0.0,0.364,0.202,0.407,0.627,0.227,0.493,1.095
136,Zimbabwe,0.246377,0.116939,0.217687,0.124906,0.108985,0.260974,0.551923,0.271220,0.510381,0.290361,0.862310,0.0,0.457,0.649,0.243,0.359,0.157,0.075,1.205
